# Polyhedral Geometry

* Author: Luca Remke
* Version: OSCAR version 0.14.0
* Last modified: January 26, 2024

This tutorial provides an introduction to polyhedral geometry in *OSCAR*.

The aim is to describe the basic constructions for convex polytopes, cones and polyhedral fans and to explain them with minimal examples.
In order to facilitate the introduction, some of the methods are not presented in their entirety. For a full description of the topic, see https://docs.oscar-system.org/stable/PolyhedralGeometry/intro/.
 
No prior knowledge of *OSCAR* is assumed.

## Content
1. [Polyhedra](#1.-Polyhedra)
   - [Polyhedra from *H*-representation](#1.A-Polyhedra-from-*H*-representation)
   - [Polyhedra from *V*-representation](#1.B-Polyhedra-from-*V*-representation)
   - [Polyhedra from existing Polyhedra](#1.C-Polyhedra-from-existing-Polyhedra)
   - [Standard Constructions](#1.D-Standard-Constructions)
   - [Geometric data of Polyhedra](#1.E-Geometric-data-of-Polyhedra)
   - [Combinatorial data of Polyhedra](#1.F-Combinatorial-data-of-Polyhedra)
   - [Groups associated to Polyhedra](#1.G-Groups-associated-to-Polyhedra)
   - [Other Functionality for Polyhedra](#1.H-Other-Functionality-for-Polyhedra)
2. [Cones](#2.-Cones)
   - [Constructors](#2.A-Constructors)
   - [Attributes and Properties](#2.B-Attributes-and-Properties)
3. [Polyhedral Fans](#3.-Polyhedral-Fans)
   - [Constructors](#3.A-Constructors)
   - [Attributes and Properties](#3.B-Attributes-and-Properties)

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.14.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2024 by The OSCAR Development Team


# 1. Polyhedra

Polyhedra can be represented as:
- a finite intersection of closed affine half-spaces (*H-representation*),
- a convex hull of finitely many points (*V-representation*).

Along these lines, *OSCAR* allows to constructor polyhedra from *H-* and *V-representations*. In addition, one can create new polyhedra from already existing polyhedra. An example of this is the Minkowski sum of two polyhedra. We will now discuss these three approaches to constructing polyhedra.

Let us mention that in *OSCAR* we can draw pictures of polyhedra. For this, simply use `visualize(P)`, where `P` is the polyhedron that you want to draw. Similarly, let us mention that it is possible to save polyhedra to files and load them afterwards:
- `save("/path/to/your/desired/data/file", P)`,
- `load("/path/to/your/desired/data/file")`.

The functionality of `visualize` extends also to cones and polyhedral fans. Saving and loading via the functions `save(filename::String, obj::Any)` and `load(filename::String)` is possible for many (if not most) objects created in *OSCAR*.

## 1.A Polyhedra from *H*-representation

There are two different ways in which polyhedra can be defined in *OSCAR* using the *H* representation:

| Method |
| :----------- |
| `Polyhedron{T}(A::AnyVecOrMat, b) where T<:scalar_types` |
| `Polyhedron{T}(I::Union{Nothing, AbstractCollection[AffineHalfspace]}, E::Union{Nothing, AbstractCollection[AffineHyperplane]} = nothing) where T<:scalar_types` |

The first method defines a polyhedron by $P = \{x \in \mathbb{R}^n \; \vert \; Ax \leq b\}$ for a matrix $A$ and a vector $b$. 
With the second method, the polyhedron is obtained by specifying the inequalities and equations of the half-spaces and hyperplanes involved in the finite section representing the polyhedron. 

The complete *H* representation of a polyhedron can be recovered using the functions `facets` and `affine_hull`.

An example is in order. For this, let us create the standard 2-simplex:

In [2]:
A = [-1 0; 0 -1; 1 1]
b = [0, 0, 1]
P1 = polyhedron(A,b)

Polyhedron in ambient dimension 2

Here is an alternative way of creating the very same polyhedron:

In [3]:
I = ([-1 0; 0 -1; 1 1], [0, 0, 1]);
P2 = polyhedron(I)

Polyhedron in ambient dimension 2

Let us verify with *OSCAR* that those polyhedra are identical:

In [4]:
P1==P2

true

To gain a better understanding of the polyhedron in question, let us also compute the facets of *P1*:

In [5]:
facets(P1)

3-element SubObjectIterator{AffineHalfspace{QQFieldElem}} over the Halfspaces of R^2 described by:
-x_1 <= 0
-x_2 <= 0
x_1 + x_2 <= 1


## 1.B Polyhedra from *V*-representation

For the *V*-representation of a polyhedron, i.e. $P = \mathrm{conv}(p_1, \ldots , p_N), p_i \in \mathbb{R}^n$, the following method is used:

| Method |
| :----------- |
| `convex_hull([::Type{T} = QQFieldElem,] V [, R [, L]]; non_redundant::Bool = false)` |

The method constructs the convex hull from vertices `V`, rays `R` and generators of the linearity space `L`. 
The arguments `R` and `L` are optional and are treated as zero if missing. 
The arguments can be given as a matrix, and the individual components must be written row by row. 
If the convex hull is to be calculated from `V` and `L` only, `R` must be defined as an empty matrix or `nothing`.

The complete *V*-representation of a polyhedron can be recovered using the functions `vertices`, `rays` and `lineality_space`.

For an example, let us revisit the 2-simplex yet again:

In [6]:
V = [0 0; 1 0; 0 1]
P3 = convex_hull(V)

Polyhedron in ambient dimension 2

Let us verify that this gives us the same polyhedron as above:

In [7]:
P2 == P3

true

Starting with a *V*-representation, the vertices are known: 

In [8]:
vertices(P3)

3-element SubObjectIterator{PointVector{QQFieldElem}}:
 [0, 0]
 [1, 0]
 [0, 1]

For a more complex example, let us look at the closed-upper half plane in $\mathbb{R}^2$:

In [9]:
V = [0 0]; R = [0 1]; L = [1 0];
H = convex_hull(V, R, L)

Polyhedron in ambient dimension 2

In [10]:
vertices(H)

0-element SubObjectIterator{PointVector{QQFieldElem}}

In [11]:
facets(H)

1-element SubObjectIterator{AffineHalfspace{QQFieldElem}} over the Halfspaces of R^2 described by:
-x_2 <= 0


## 1.C Polyhedra from existing Polyhedra

Polyhedra can be produced through operations on other polyhedra:

| Method | Description |
| :----------- | :----------- |
| `+(P::Polyhedron, Q::Polyhedron)` | Minkowski sum $P+Q = \{x+y \; \vert \; x \in P, y \in Q\}$ von `P` und `Q`. |
| `*(k::Int, Q::Polyhedron)` | Scaled polyhedron  $kQ = \{kx \; \vert \; x \in Q\}$. |
| `*(P::Polyhedron, Q::Polyhedron)` | Cartesian product of `P` and `Q`. |
| `bipyramid(P::Polyhedron, z::Number = 1, z_prime::Number = -z)` | Bipyramid over a pointed polyhedron `P`.  |
| `intersect(P::Polyhedron, Q::Polyhedron)` | Intersection $P \cap Q$ of `P` and `Q`. |
| `pyramid(P::Polyhedron, z::Number = 1)` | Pyramid over `P` with distance `z` between the vertex barycenter of `P` and the top of the pyramid. |
| `convex_hull(P::Polyhedron, Q::Polyhedron)` | Convex hull of `P` and `Q`. |

For an example, consider the Minkowski sum $\Delta_2 + \Delta_2$ of the standard 2-simplexes is given by $\{(0,0), (1,0), (0,1), (1,1), (2,0),(0,2)\}$ and thus defined by the vertices $(0,0)$, $(2,0)$ and $(0,2)$. 
This is exactly the polyhedron $2\Delta_2$ that we obtain by scaling the standard 2-simplex by $2$. 
Moreover, since $(\tfrac{1}{3}, \tfrac{1}{3})$ is the centroid of $\Delta_2$, we get a pyramid with vertex $(\tfrac{1}{3}, \tfrac{1}{3}, 1)$.

In [12]:
S1 = +(simplex(2), simplex(2)) #or: simplex(2) + simplex(2)
S2 = *(2, simplex(2)) #or: 2*simplex(2)
S3 = *(simplex(2), simplex(2)) #or: simplex(2) * simplex(2)
S4 = pyramid(simplex(2))

[vertices(S1), vertices(S2), vertices(S3), vertices(S4)]

4-element Vector{SubObjectIterator{PointVector{QQFieldElem}}}:
 [[0, 0], [2, 0], [0, 2]]
 [[0, 0], [2, 0], [0, 2]]
 [[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1], [0, 1, 0, 0], [0, 1, 1, 0], [0, 1, 0, 1]]
 [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1//3, 1//3, 1]]

## 1.D Standard Constructions

Many known literature and standard constructions for polyhedra as supported in *OSCAR*. The supported constructions include the following:

| Function | Description |
| :----------- | :----------- |
| `archimedean_solid(s::String)` | Archimedean solid with the name given by String `s`, see [here](https://docs.oscar-system.org/stable/PolyhedralGeometry/Polyhedra/constructions/). |
| `birkhoff(n::Integer, even::Bool = false)` | Birkhoff polytop of dimension `n`$^2$. |
| `catalan_solid(s::String)` | Catalan solid with the name given by String `s`, see [here](https://docs.oscar-system.org/stable/PolyhedralGeometry/Polyhedra/constructions/). |
| `cross_polytope([::Type{T} = QQFieldElem,] d::Int [,n::Rational])` | `d`-dimensional cross polytope around the origin with vertices located at $\pm e_i$ for each unit vector $e_i$ of $\mathbb{R}^d$, scaled by `n`. |
| `cube([::Type{T} = QQFieldElem,] d::Int , [l::Rational = -1, u::Rational = 1])` | $[l,u]$-cube in dimension `d`. |
| `cyclic_polytope(d::Int, n::Int)` | Cyclic polytope that is the convex hull of `n` points on the moment curve in dimension `d`. |
| `del_pezzo_polytope(d::Int)` | `d`-dimensional del Pezzo polytop. |
| `dodecahedron()` | Regular dodecahedron, one of the Platonic solids. |
| `fano_simplex(d::Int)` | A lattice simplex such that the origin is the unique interior lattice point. |
| `fractional_cut_polytope(G::Graph{Undirected})` | The fractional cut polytope of the graph `G`. |
| `fractional_matching_polytope(G::Graph{Undirected})` | The fractional matching polytope of the graph `G`. |
| `gelfand_tsetlin(lambda::AbstractVector)` | Gelfand Tsetlin polytope indexed by a weakly decreasing vector `lambda`. |
| `icosahedron()` | Regular icosahedron, one of the Platonic solids. |
| `johnson_solid(i::Int)` | `i`-th proper Johnson solid. |
| `newton_polytope(poly::Polynomial)` | Newton polytope of the multivariate polynomial `poly`. |
| `orbit_polytope(V::AbstractCollection[PointVector], G::PermGroup)` | Convex hull of the orbit of one or several points (given row-wise in `V`) under the action of `G`. |
| `platonic_solid(s::String)` | Platonic solid with the name given by String `s`, see [here](https://docs.oscar-system.org/stable/PolyhedralGeometry/Polyhedra/constructions/). |
| `rand_spherical_polytope([rng::AbstractRNG,] d::Int, n::Int;` <br> `distribution=:uniform, precision=nothing, seed=nothing)` | The convex hull of `n` points on the unit sphere in $\mathbb{R}^d$. |
| `regular_24_cell()` | Regular 24-cell, one out of three exceptional regular 4-polytopes. |
| `regular_120_cell()` | Regular 120-cell, one out of three exceptional regular 4-polytopes. |
| `regular_600_cell()` | Regular 600-cell, one out of three exceptional regular 4-polytopes. |
| `simplex([::Type{T} = QQFieldElem,] d::Int [,n::Rational])` | Simplex which is the convex hull of the standard basis vectors along with the origin in $\mathbb{R}^d$, scaled by `n`. |
| `tetrahedron()` | Regular tetrahedron, one of the Platonic solids. |


## 1.E Geometric data of Polyhedra

Of course, there is geometric data associated to polyhedra. Among others, the following is supported in *OSCAR*:

| Method | Description |
| :----------- | :----------- |
| `facets(as::Type{T} = AffineHalfspace, P::Polyhedron)` | Facets of `P` in the format defined by `as` (`Halfspace` (Default), `Polyhedron` or `Pair`). |
| `vertices(P::Polyhedron)` | Iterator over the vertices of a polyhedron `P` as points. |
| `rays(P::Polyhedron)` | Minimal set of generators of the cone of unbounded directions of `P` (i.e. its rays) as points. |
| `rays_modulo_lineality(as, P::Polyhedron)` | Rays of the recession cone of `P` up to lineality as a `NamedTuple` with two iterators. |
| `minimal_faces(as, P::Polyhedron)` | Minimal faces of a polyhedron as a `NamedTuple` with two iterators. |
| `affine_hull(P::Polytope)` | (Affine) hyperplanes generating the affine hull of `P`.  |
| `ambient_dim(P::Polyhedron)` | Ambient dimension of `P`. |
| `dim(P::Polyhedron)` | Dimension of `P`. |
| `codim(P::Polyhedron)` | Codimension of `P`. |
| `is_bounded(P::Polyhedron)` | Checks whether `P` is bounded. |
| `is_feasible(P::Polyhedron)` | Checks whether `P` is non-empty. |
| `is_fulldimensional(P::Polyhedron)` | Checks whether `P` is full-dimensional. |
| `is_lattice_polytope(P::Polyhedron{QQFieldElem})` | Checks whether `P` is bounded and has integral vertices. |
| `lineality_dim(P::Polyhedron)` | Dimension of the largest affine subspace contained in `P`. |
| `lineality_space(P::Polyhedron)` | Matrix whose row span is the lineality space of `P`. |
| `recession_cone(P::Polyhedron)` | Recession cone of `P`. |
| `relative_interior_point(P::Polyhedron)` | A point in `P` not contained in any facet. |

Here is an example:

In [13]:
# geometric data of the standard 2-simplex
S = simplex(2)

[
    vertices(S), facets(Pair, S), 
    (ambient_dim(S), dim(S), codim(S)),
    (is_bounded(S), is_feasible(S), is_fulldimensional(S)),
    lineality_space(S),
    relative_interior_point(S)    
]

6-element Vector{Any}:
 PointVector{QQFieldElem}[[0, 0], [1, 0], [0, 1]]
 Pair{Matrix{QQFieldElem}, QQFieldElem}[[-1 0] => 0, [0 -1] => 0, [1 1] => 1]
 (2, 2, 0)
 (true, true, true)
 0-element SubObjectIterator{RayVector{QQFieldElem}}
 QQFieldElem[1//3, 1//3]

## 1.F Combinatorial data of Polyhedra

Similarly, combinatorial data associated to polyhedra is available in *OSCAR*:

| Method | Description |
| :----------- | :----------- |
| `nfacets(P::Polyhedron)` | Number of facets of `P`. |
| `nvertices(P::Polyhedron)` | Number of vertices of `P`. |
| `f_vector(P::Polyhedron)` | Vector $(f_1, ..., f_{\dim(PF)-1})$, where $f_i$ is the number of faces of `P` of dimension $i$. |
| `g_vector(P::Polyhedron)` | Toric $g$-vector of a (bounded) polytope, defined by $g_0=1$ and $g_k = h_k - h_{k-1}$ for $1 \leq k \leq \lceil (d+1)/2 \rceil$, where $h$ is the $h$-vector and $d = \dim P$. |
| `h_vector(P::Polyhedron)` | Toric $h$-vector of a (bounded) polytope; for simplicial polytopes this is a linear transformation of the $f$-vector. |

Let us demonstrate this functionality:

In [14]:
# combinatorial data of the standard 2-simplex
S = simplex(2)
[
    (nvertices(S), nfacets(S)),
    f_vector(S),
    g_vector(S),
    h_vector(S)
]

4-element Vector{Any}:
 (3, 3)
 ZZRingElem[3, 3]
 ZZRingElem[1, 0]
 ZZRingElem[1, 1, 1]

## 1.G Groups associated to Polyhedra

Finally, there are even groups associated to polyhedra. The following are supported:

| Method | Description |
| :----------- | :----------- |
| `linear_symmetries(P::Polyhedron)` | Group of linear symmetries on the vertices of a polyhedron given as permutations of the vertices (or rather vertex indices) of `P`. |
| `combinatorial_symmetries(P::Polyhedron)` | Combinatorial symmetries (i.e., automorphisms of the face lattice) of `P` given as permutations of the vertices (or rather vertex indices) of `P`. |
| `automorphism_group(P::Polyhedron; type = :combinatorial, action = :all)` | Group of automorphisms of a polyhedron `P`. The parameters are `:combinatorial` or `:linear` resp. `:all`, `:on_vertices` or `:on_facets`. |
| `automorphism_group_generators(P::Polyhedron; type = :combinatorial, action = :all)` | Generators of the group of automorphisms of a polyhedron `P` (parameter as above). |
| `automorphism_group(IM::IncidenceMatrix; action = :all)` | Group of automorphisms of an IncidenceMatrix `IM`.  |
| `automorphism_group_generators(IM::IncidenceMatrix; action = :all)` | Generators of the group of automorphisms of an IncidenceMatrix `IM`. The parameters for `action` are `:all` (Default), `:on_cols` and `:on_rows` for only the generators of the permutation action on the columns resp. rows. |

For an example, let us return to the standard 2-simplex and compute the group of its linear symmetries as well as its automorphism group:

In [15]:
S = simplex(2)

Polyhedron in ambient dimension 2

In [16]:
G1 = linear_symmetries(S)

Permutation group of degree 3

In [17]:
elements(G1)

6-element Vector{PermGroupElem}:
 ()
 (2,3)
 (1,2)
 (1,2,3)
 (1,3,2)
 (1,3)

In [18]:
G2 = automorphism_group(S)

Dict{Symbol, PermGroup} with 2 entries:
  :on_vertices => Permutation group of degree 3
  :on_facets   => Permutation group of degree 3

## 1.H Other Functionality for Polyhedra

There is yet more functionality avaialble for polyhedra. This includes the following:

| Method | Description |
| :----------- | :----------- |
| `all_triangulations(P::Polyhedron)` | All triangulations on the points given as the rows of `P`. |
| `boundary_lattice_points(P::Polyhedron{QQFieldElem})` | Integer points contained in the boundary of the bounded polyhedron `P`. |
| `in(v::AbstractVector, P::Polyhedron)` | Checks whether the vector `v` is contained in the polyhedron `P`. |
| `issubset(P::Polyhedron, Q::Polyhedron)` | Checks whether `P` is a subset of the polyhedron `Q`. |
| `ehrhart_polynomial(R::QQMPolyRing, P::Polyhedron{QQFieldElem})` | Computes the Ehrhart polynomial of `P` and returns it as a polynomial in `R`. |
| `h_star_polynomial(R::QQMPolyRing, P::Polyhedron)` | Computes the $h^\ast$ polynomial of `P` (and optionally returns it as a polynomial in `R`). |
| `interior_lattice_points(P::Polyhedron{QQFieldElem})` | Integer points contained in the interior of the bounded polyhedron `P`. |
| `is_normal(P::Polyhedron{QQFieldElem})` | Checks whether `P` is normal. |
| `is_simple(P::Polyhedron)` | Checks whether `P` is simple. |
| `is_smooth(P::Polyhedron{QQFieldElem})` | Checks whether `P` is smooth. |
| `is_very_ample(P::Polyhedron{QQFieldElem})` | Checks whether `P` is very ample. |
| `lattice_points(P::Polyhedron{QQFieldElem})` | Integer points contained in the bounded polyhedron `P`. |
| `lattice_volume(P::Polyhedron{QQFieldElem})` | Lattice volume of `P`. |
| `normalized_volume(P::Polyhedron)` | The (normalized) volume of `P`. |
| `polarize(P::Polyhedron)` | Polar dual of `P`. |
| `project_full(P::Polyhedron)` | Project the polyhedron `P` down such that it becomes full dimensional in the new ambient space. |
| `print_constraints(A::AnyVecOrMat, b::AbstractVector; trivial::Bool = false, numbered::Bool = false)` <br> `print_constraints(P::Polyhedron; trivial::Bool = false, numbered::Bool = false)` | Pretty print the constraints given by $P(A,b) = \{x \; \vert \; Ax \leq b\}$. Trivial inequalities are included if `trivial` is set to `true`. |
| `regular_triangulations(pts::AbstractCollection[PointVector]; full=false)` <br> `regular_triangulations(P::Polyhedron)` | All regular triangulations on the points given as the rows of `pts` resp. all regular triangulations that can be formed using the vertices of the given bounded and full-dimensional polytope `P`. |
| `secondary_polytope(P::Polyhedron)` | Secondary polytope of a polyhedron `P`. |
| `solve_ineq(A::fmpz_mat, b::fmpz_mat)` | Solves $Ax \leq b$, assuming finite solution set. |
| `solve_mixed(A::fmpz_mat, b::fmpz_mat, C::fmpz_mat, d::fmpz_mat)` | Solves $Ax = b$ under $Cx \geq d$, assuming finite solution set. |
| `solve_non_negative(A::fmpz_mat, b::fmpz_mat)` | Solves $Ax = b$ under $Cx \geq 0$, assuming finite solution set.  |
| `support_function(P::Polyhedron; convention::Symbol = :max)` | Produce a function $h(\omega) = \max(\dot(x,\omega) \; \vert \; x \in P\}$. $\max$ may be changed to $\min$ by setting `convention = : min`. |
| `volume(P::Polyhedron)` | (Euclidean) volume of `P`. |

For an example, look once again at the standard 2-simplex $\Delta_2$. It contains the lattice points $(0,0)$, $e_1 = (1,0)$ and $e_2 = (0,1)$, which are all edge points. So it contains the vector $(1,0)$ but not the vector $(1,1)$. This polyhedron $\Delta_2$ is normal, smooth, simple and very ample. The volume of $\Delta_2$ is $\tfrac{1}{2} \cdot |e_1| \cdot |e_2| = \tfrac{1}{2}$ and the lattice volume is $\sqrt{\det(e_1,e_2)} = 1$. Since the origin $(0,0)$ is not an interior point of $\Delta_2$, the dual of the standard 2-simplex is an unbounded polyhedron with rays $e_1$ and $e_2$. Let us recompute these results with *OSCAR*:

In [19]:
S = simplex(2)

Polyhedron in ambient dimension 2

In [20]:
boundary_lattice_points(S)

3-element SubObjectIterator{PointVector{ZZRingElem}}:
 [0, 0]
 [0, 1]
 [1, 0]

In [21]:
interior_lattice_points(S)

0-element SubObjectIterator{PointVector{ZZRingElem}}

In [22]:
contains(S, [0, 1])

true

In [23]:
contains(S, [1, 1])

false

In [24]:
is_normal(S)

true

In [25]:
is_simple(S)

true

In [26]:
is_smooth(S)

true

In [27]:
is_very_ample(S)

true

In [28]:
lattice_volume(S)

1

In [29]:
normalized_volume(S)

1

In [30]:
volume(S)

1//2

In [31]:
S_dual = polarize(S)

Polyhedron in ambient dimension 2

In [32]:
rays(S_dual)

2-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [0, 1]

In [33]:
is_bounded(S_dual)

false

# 2. Cones

Although polyhedral cones are special cases of polyhedra, they are treated as a separate type in OSCAR. 
For this reason, some of the methods of polyhedra will also appear here.

## 2.A Constructors

The standard way to construct a polyhedral cone is to define the positive hull of its rays `R`, with lineality given by `L`.

| Method |
| :----------- |
| `positive_hull([::Type{T} = QQFieldElem,] R::AbstractCollection[RayVector] [, L::AbstractCollection[RayVector]]; non_redundant::Bool = false) where T<:scalar_types` |

Here, `R` is given row-wise as representative vectors, with lineality generated by the rows of `L`, i.e. the cone consists of all positive linear combinations of the rows of `R` plus all linear combinations of the rows of `L`.

In addition, the following functions can be used to construct a polyhedral cone:

| Method |
| :----------- |
| `cone_from_inequalities([::Type{T} = QQFieldElem,] I::AbstractCollection[LinearHalfspace] [, E::AbstractCollection[LinearHyperplane]]; non_redundant::Bool = false)` | 
| `cone_from_equations([::Type{T} = QQFieldElem,] E::AbstractCollection[LinearHyperplane]; non_redundant::Bool = false)` |

These methods construct the (convex) cone defined by $\{x \; \vert \; Ix \leq 0, \, Ex=0\}$ resp. $\{x \; \vert \; Ex=0\}$.
To avoid unnecessary redundancy checks, set `non_redundant` to `true`, if the given description contains no redundant rows.

The construction of the secondary cone is done by:

| Method |
| :----------- |
| `secondary_cone(SOP::SubdivisionOfPoints)` |

Here is an example:

In [34]:
R = [2 -1; 0 1]
C = positive_hull(R)

Polyhedral cone in ambient dimension 2

## 2.B Attributes and Properties

| Method | Description |
| :----------- | :----------- |
| `ambient_dim(C::Cone)` | Ambient dimension of  `C`. |
| `in(v::AbstractVector, C::Cone)` | Checks whether the vector `v` is contained in the cone `C`. |
| `issubset(C0::Cone, C1::Cone)` | Checks whether `C0` is a subset of the cone `C1`. |
| `f_vector(C::Cone)` | Vector $(f_1, ..., f_{dim(PF)-1})$, where $f_i$ is the number of faces of `C` of dimension $i$. |
| `hilbert_basis(C::Cone{QQFieldElem})` | Hilbert basis of a pointed cone `C` as the rows of a matrix. |
| `codim(C::Cone)` | Codimension of `C`. |
| `dim(C::Cone)` | Dimension of `C`. |
| `polarize(C::Cone)` | Polar dual of `P`. |
| `intersect(C0::Cone{T}, C1::Cone{T}) where T<:scalar_types` | Intersection $C0 \cap C1$ of `C0` and `C1`. |
| `is_pointed(C::Cone)` | Determines whether `C` is pointed. |
| `is_fulldimensional(C::Cone)` | Determines whether `C` is full-dimensional. |
| `lineality_dim(C::Cone)` | Dimension of the largest linear subspace contained in `C`. |
| `lineality_space(C::Cone)` | Basis of the lineality space of `C`. |
| `nfacets(C::Cone)` | Number of facets of a cone `C`. |
| `nrays(C::Cone)` | Number of rays of `C`. |
| `rays(C::Cone)` | Rays of `C`. |
| `rays_modulo_lineality(as, C::Cone)` | Rays of the cone of `C` up to lineality as a `NamedTuple` with two iterators.  |

We demonstrate this with a simple example. The cone $\sigma \subset \mathbb{Q}^2$ is pointed and full-dimensional with the two edges $2e_1 - e_2$ and $e_2$, which are also facets. The dual $\sigma^\vee$ is generated by the rays $e_1$ and $e_1 + 2e_2$.

In [35]:
R = [2 -1; 0 1]
C = positive_hull(R)

Polyhedral cone in ambient dimension 2

In [36]:
ambient_dim(C)

2

In [37]:
dim(C)

2

In [38]:
codim(C)

0

In [39]:
is_pointed(C)

true

In [40]:
is_fulldimensional(C)

true

In [41]:
nrays(C)

2

In [42]:
nfacets(C)

2

In [43]:
rays(polarize(C))

2-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [1, 2]

# 3. Polyhedral Fans

## 3.A Constructors

To construct a polyhedral fan, you must pass the rays of each cone in the fan, along with an IncidenceMatrix encoding which rays generate which cones:

| Method |
| :----------- |
| `PolyhedralFan{T}(Rays, Cones; non_redundant = false) where T<:scalar_types` |

The arguments of `PolyhedralFan` are:
- `Rays::AbstractCollection[RayVector]` : Rays generating the cones of the fan; encoded row-wise as representative vectors.
- `Cones::IncidenceMatrix` : An incidence matrix defined by 1 at position $(i,j)$ if cone $i$ has ray $j$ as an extremal ray and 0 otherwise.

A polyhedral fan formed from rays and cones made of these rays. The cones are given as an IncidenceMatrix, where the columns represent the rays and the rows represent the cones.

The following method constructs a polyhedral fan with a group action:

| Method |
| :----------- |
| `polyhedral_fan_from_rays_action(::Type{T}, Rays::AbstractCollection[RayVector], MC_reps::IncidenceMatrix, perms::AbstractVector{PermGroupElem}) where T<:scalar_types` |

with arguments:
- `Rays`: The rays of the fan
- `MC_reps`: IncidenceMatrix whose rows give the indices of the rays forming representatives of the maximal cones under the group action.
- `perms`: A vector of permutations PermGroupElem that form generators of the group acting on the rays of the fan.


The normal fan and the face fan of a polyhedron `P` can also be constructed:

| Method |
| :----------- |
| `normal_fan(P::Polyhedron)` |
| `face_fan(P::Polyhedron)` |

For an example, consider the fan $\Sigma_{\mathbb{P}^2}$ associated with the projective plane $\mathbb{P}^2$. This fan has ray generators $e_1$, $e_2$ and $-e_1-e_2$.

In [44]:
R = [1 0; 0 1; -1 -1]
IM = IncidenceMatrix([[1, 2], [2, 3], [1, 3]])
PF = PolyhedralFan(R, IM)

Polyhedral fan in ambient dimension 2

## 3.B Attributes and Properties

| Methode | Rückgabe |
| :----------- | :----------- |
| `ambient_dim(PF::PolyhedralFan)` | Ambient dimension of `PF`. |
| `dim(PF::PolyhedralFan)` | Dimension of `PF`. |
| `f_vector(PF::PolyhedralFan)` | Vector $(f_1, ..., f_{dim(PF)-1})$, where $f_i$ is the number of faces of `PF` of dimension $i$. |
| `is_complete(PF::PolyhedralFan)` | Determines whether `PF` is complete. |
| `is_pointed(PF::PolyhedralFan)` | Determines whether `PF` is pointed. |
| `is_regular(PF::PolyhedralFan)` | Determines whether `PF` is regular. |
| `is_simplicial(PF::PolyhedralFan)` | Determines whether `PF` is simpicial. |
| `is_smooth(PF::PolyhedralFan{fmpq})` | Determines whether `PF` is smooth. |
| `lineality_dim(PF::PolyhedralFan)` | Dimension of the largest linear subspace of `PF`. |
| `lineality_space(PF::PolyhedralFan)` | Non-redundant matrix whose rows are generators of the lineality space of  `PF`. |
| `maximal_cones(PF::PolyhedralFan)` | Maximal cones of  `PF`. |
| `cones(PF::PolyhedralFan, cone_dim::Int)` | Iterator over the cones of `PF` of dimension `cone_dim`. |
| `cones(PF::PolyhedralFan)` | Ray indices of all non-zero-dimensional cones in a polyhedral fan `PF`. |
| `n_maximal_cones(PF::PolyhedralFan)` | Number of maximal cones of `PF`. |
| `n_cones(PF::PolyhedralFan)` | Number of cones of `PF`. |
| `nrays(PF::PolyhedralFan)` | Number of rays of `PF`. |
| `rays(PF::PolyhedralFan)` | Rays of `P`. |
| `rays_modulo_lineality(as, F::PolyhedralFan)` | Rays of the polyhedral fan `F` up to lineality as a `NamedTuple` with two iterators. |
| `primitive_collections(PF::PolyhedralFan)` | Primitive collections of `PF`. |
| `starsubdivision(PF::PolyhedralFan, n::Int)` | Star subdivision of a polyhedral fan `PF` at its `n`-th maximal torus orbit. |
| `*(PF1::PolyhedralFan, PF2::PolyhedralFan)` | Cartesian/direct product of two polyhedral fans `PF1` and `PF2`. |

### Example

The fan $\Sigma_{\mathbb{P}^2} \subset \mathbb{Q}^2$ consists of faces $\{0\}$, three $1$-dimensional rays $\tau_{ij} = \sigma_i \cap \sigma_j$ and three $2$-dimensional facets $\sigma_0$, $\sigma_1$ and $\sigma_2$, all of which are strongly-convex. Further, the fan $\sigma_{\mathbb{P}^2}$ is complete, smooth and regular. It is the normal fan of the standard 2-simplex $\Delta_2$.

In [45]:
R = [1 0; 0 1; -1 -1]
IM = IncidenceMatrix([[1, 2], [2, 3], [1, 3]])
PF = PolyhedralFan(R, IM)

Polyhedral fan in ambient dimension 2

In [46]:
ambient_dim(PF)

2

In [47]:
dim(PF)

2

The f_vector tells us how many cones of dimensions 1, 2, 3, etc. exist in a polyhedral fan. For *PF*, there are three 1-dimensional and three 2-dimensional cones and no higher dimensional cones. So the answer should be (3,3):

In [48]:
f_vector(PF)

2-element Vector{ZZRingElem}:
 3
 3

In [49]:
is_pointed(PF)

true

In [50]:
is_complete(PF)

true

In [51]:
is_smooth(PF)

true

In [52]:
is_simplicial(PF)

true

In [53]:
is_regular(PF)

true

In [54]:
rays(PF)

3-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [0, 1]
 [-1, -1]

In [55]:
rays(normal_fan(simplex(2)))

3-element SubObjectIterator{RayVector{QQFieldElem}}:
 [1, 0]
 [0, 1]
 [-1, -1]